In [1]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from langchain.llms import OpenAI
from langchain import HuggingFaceHub,LLMChain
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import pandas as pd
from collections import Counter
from io import StringIO
import streamlit as st
import re
import os
import openai
import ast
import json
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
# Decorator for automatic retry requests

from pydantic import BaseModel, Field, conlist
from typing import List, Optional, Tuple
class OutputResult(BaseModel):
    key: conlist(str, min_length=3, max_length=5) = Field(description="The key with the story parameters. Must contain between 3 and 5 parameters")
    story:str = Field(description="The generated story for the given key")
    
@retry(
    retry = retry_if_exception_type((openai.APIError, openai.APIConnectionError,  openai.Timeout, ValueError, SyntaxError,KeyError)),
    # Function to add random exponential backoff to a request
    wait = wait_random_exponential(multiplier = 1, max = 60),
    stop = stop_after_attempt(10)
)
def run_llm_chain(hub_chain,user_input,parser):    
    output =hub_chain.run(input=user_input)        
    parsed_result = parser.parse(output)                   
    return parsed_result

In [2]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub,LLMChain
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import pandas as pd
from collections import Counter
from io import StringIO
import streamlit as st
import re
import os
import openai
import ast
import json
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator


def createDataset(iterations, key, story_type, instructions,temp) -> pd.DataFrame:
    import os
    import openai
    import ast
    from langchain.chat_models import ChatOpenAI
# initialize the models
    openai.api_key = os.getenv("OPENAI_API_KEY")
    openai = ChatOpenAI(             
        model_name="gpt-3.5-turbo",
        openai_api_key=openai.api_key,
        temperature=temp
    )  

    
    examples = [          
            {
                "input": """Generate a tuple with first part a key built like this [age,gender, superpower], 
                and the value in the tuple will be an entire story of maximum 100 words with detailed description for a super-hero with the given age, of the given gender and with the given superpower """,
                "output": OutputResult.model_validate({
                    "key": ["18", "man", "invisibility"],
                    "story": """A 18 year old man, tall with a strong yet athletic build. Noir eyes and light brown hair that seems to be a reflection of the warmth of his personality. 
                    His superpower of invisibility make him silent, 
                    introspective and observant. He knows when to be seen and when to remain invisible in the background; like a silent guardian protecting those around him. With a strong sense of justice and power,
                    he is an invaluable asset to those he holds near and dear. His kind and compassionate spirit give him an aura of protectiveness, making him a person of strength and courage in difficult moments."""            
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            },
            {
                "input": """Generate a tuple with first part a key built like this [product ,theme, details], 
                and the value in the tuple will be a gingle of maximum 100 words with commercial for the given product, in the given theme incorporating the provided details.""",
                "output": OutputResult.model_validate({
                    "key": ["Whiskers", "happy", "cat food-holiday season price reductions-great for your cat"],
                    "story": "We are so happy to announce holiday discounts for the best cat food outhere! For happy and healthy cat choose Whiskers! Meow!"            
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),                
            },
             {
                "input": """Generate a tuple with first part a key built like this [fictional character ,location, adventure], 
                and the value in the tuple will be a story of maximum 100 words describing an adventure of the given fictional character in the provided location.""",
                "output": OutputResult.model_validate({
                    "key": ["Baba Yaga", "Asia", "getting no respect"],
                    "story": """Once upon a time Baba Yaga wondered far far away from her home and ended up in remote Hokkaido island. 
                    She was used to locals showing her great respect out of fear and also because she was always one of the pillars of Slavic culture. 
                    But in Hokkaido the locals knew nothing about her, and she was very disappointed because they have shown her no respect. Eventually she decided there is no place like home and went back"""            
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),     
            },
        ]


    

    # create a example template
    example_template = """
        User: {input}
        AI: {output}
    """
    # create a prompt example from above template
    example_prompt = PromptTemplate(
        input_variables=["input", "output"],
        template=example_template
    )

    parser = PydanticOutputParser(pydantic_object=OutputResult)

    # now break our previous prompt into a prefix and suffix
    # the prefix is our instructions    
    prefix = """You are a helpful assistant great in story telling. You follow the given instructions in a precise manner. 
    You need to generate a dataset where the key would be generated values string representing the story parameters according to the user given instructions, and the value will be a story written given this key. 
    Transform the output into structured object given those instructions: {format_instructions} Here are a few examples on how to generate the content of the dataset:
    """

    # and the suffix our user input and output indicator
    suffix = """
    User: {input}
    AI:"""


    # now create the few shot prompt template
    few_shot_prompt_template = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix=prefix,
        suffix=suffix,
        input_variables=["input"],
        example_separator="\n\n",
        partial_variables={"format_instructions": parser.get_format_instructions()},    
    )

    f_prompt = "Generate a tuple with first part a key built like this [{key}], and the value in the tuple will be an entire {type} of maximum 100 words .{instructions}."
    user_input = f_prompt.format(key=key, type=story_type,instructions = instructions)
    df = pd.DataFrame()
    for i in range(iterations):
            hub_chain = LLMChain(prompt=few_shot_prompt_template,llm=openai,verbose=True)              
            output  = run_llm_chain(hub_chain,user_input,parser)                                     
           
           
            generated_key = ", ".join(output.key) if output.key else 'Not specified'
            print('generated_key:',generated_key)

            generated_value = output.story if output.story else 'Not specified'
            print('generated_value:',generated_value)      
            
            

            # Access and print the key-value pairs
            
            new_row = {
            'keywords':key, 
            'story_type':story_type, 
            'instructions':instructions,         
            'generated_key':generated_key,
            'generated_value': generated_value
            }
            new_row = pd.DataFrame([new_row])
            df = pd.concat([df, new_row], axis=0, ignore_index=True)
    
    return df

In [3]:
news_coldTemp = createDataset(5, 'news agency, location, news item', 'news item',
                           """The news item will be a report of the given news agency regarding the provided location
                             and revolving around the news item which takes place in that location""",0.7)
news_coldTemp

c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  wa



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant great in story telling. You follow the given instructions in a precise manner. 
    You need to generate a dataset where the key would be generated values string representing the story parameters according to the user given instructions, and the value will be a story written given this key. 
    Transform the output into structured object given those instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"key": {"description": "The key with the story parameters. Must contain betwee

,keywords,story_type,instructions,generated_key,generated_value
0,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global Times, China, COVID-19 vaccine distribu...",Global Times reports that China has successful...
1,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York City, mysterious disappe...","In a shocking turn of events, a famous artist ..."
2,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York, Earthquake","In a shocking turn of events, New York City wa..."
3,"news agency, location, news item",news item,The news item will be a report of the given ne...,"The Daily Gazette, New York City, Alien Invasion",Breaking news from The Daily Gazette in New Yo...
4,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York City, alien invasion",Breaking news from New York City! Reports are ...


In [4]:
news_mildTemp = createDataset(5, 'news agency, location, news item', 'news item',
                           """The news item will be a report of the given news agency regarding the provided location
                             and revolving around the news item which takes place in that location""",1.0)
news_mildTemp



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant great in story telling. You follow the given instructions in a precise manner. 
    You need to generate a dataset where the key would be generated values string representing the story parameters according to the user given instructions, and the value will be a story written given this key. 
    Transform the output into structured object given those instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"key": {"description": "The key with the story parameters. Must contain betwee

,keywords,story_type,instructions,generated_key,generated_value
0,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York City, major protest in T...","In a breaking news report from Global News, a ..."
1,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York City, terrorist attack a...","In a shocking turn of events, a terrorist atta..."
2,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York City, Natural Disaster",Global News reports from the heart of New York...
3,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Global News, New York, earthquake in Manhattan",Breaking News from Global News: An unexpected ...
4,"news agency, location, news item",news item,The news item will be a report of the given ne...,"Daily Globe, New York, Alien Invasion",Breaking news from the Daily Globe! Reports ar...


In [ ]:
news_highTemp = createDataset(5, 'news agency, location, news item', 'news item',
                           """The news item will be a report of the given news agency regarding the provided location
                             and revolving around the news item which takes place in that location""",2.0)
news_highTemp

In [ ]:
another_df.to_csv("../datasets/workout_generated_dataset.csv")